In [1]:
import nltk
import threading
import queue
import feedparser
import uuid

In [2]:
threads = []
queues = [queue.Queue(), queue.Queue()]

In [3]:
def extractWords():
    url = 'https://timesofindia.indiatimes.com/rssfeeds/1081479906.cms'
    feed = feedparser.parse(url)
    for entry in feed['entries'][:5]:
        text = entry['title']
        if 'ex' in text:
            continue
        words = nltk.word_tokenize(text)
        data = {'uuid': uuid.uuid4(), 'input': words}
        queues[0].put(data, True)
        print(">> {} : {}".format(data['uuid'], text))

In [4]:
def extractPOS():
    while True:
        if queues[0].empty():
            break
        else:
            data = queues[0].get()
            words = data['input']
            postags = nltk.pos_tag(words)
            queues[0].task_done()
            queues[1].put({'uuid': data['uuid'], 'input': postags}, True)

In [5]:
def extractNE():
    while True:
        if queues[1].empty():
            break
        else:
            data = queues[1].get()
            postags = data['input']
            queues[1].task_done()
            chunks = nltk.ne_chunk(postags, binary=False)
            print("  << {} : ".format(data['uuid']), end = '')
            for path in chunks:
                try:
                    label = path.label()
                    print(path, end=', ')
                except:
                    pass
            print()

In [6]:
def runProgram():
    e = threading.Thread(target=extractWords())
    e.start()
    threads.append(e)

    p = threading.Thread(target=extractPOS())
    p.start()
    threads.append(p)

    n = threading.Thread(target=extractNE())
    n.start()
    threads.append(n)

    queues[0].join()
    queues[1].join()

    for t in threads:
        t.join()

In [7]:
runProgram()

>> 3d22b0f5-b6c2-4212-8862-014273cdd37d : Kartik Aaryan papped with fans in Lucknow
>> da3e5794-4d0c-4d23-abc9-678a6f2007b5 : Kunal, Shahid, Ishaan off on a biking trip
>> dd06f9df-275f-4633-b0cd-9482f34924cb : Sushmita: Becoming mother a wisest decision
>> c20c1fe5-7969-42ac-a66f-312e3b4ea59f : Mira slams online retailer for plastic packs
>> 19c2180c-f94a-487b-83ae-22453192e783 : Pic:Tiger resonates with people stuck in rains
  << 3d22b0f5-b6c2-4212-8862-014273cdd37d : (PERSON Kartik/NNP), (PERSON Aaryan/NNP), (GPE Lucknow/NNP), 
  << da3e5794-4d0c-4d23-abc9-678a6f2007b5 : (GPE Kunal/NNP), (PERSON Shahid/NNP), (PERSON Ishaan/NNP), 
  << dd06f9df-275f-4633-b0cd-9482f34924cb : (GPE Sushmita/NN), 
  << c20c1fe5-7969-42ac-a66f-312e3b4ea59f : (GPE Mira/NNP), 
  << 19c2180c-f94a-487b-83ae-22453192e783 : (PERSON Tiger/NN), 
